In [ ]:
from scipy import *
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import matplotlib.animation
import PlanktonSignaling.basics as PS
import PlanktonSignaling.BlobDetection as BD
import PlanktonSignaling.Deposition as DP
import profile

%matplotlib notebook

#%load_ext Cython

In [ ]:
meshsize = 40
print('Mesh length scale: {0:8.2e}'.format(1/meshsize))
Swimmers = PS.Plankton(DP.constantDep,N = meshsize,depMaxStr=1.0e-4,depVar=4.0e-4,k=0.02,speed=0.05,
                    lambda0=3,kappa=6.4e-3,beta=1,depTransWidth=0.001,depThreshold=0.08)

def initial_conditions(x,y):
    return(0*x)

In [ ]:
Swimmers.SetIC(initial_conditions)

pos = [array([0.1,0.1])]
aggreg = [array([0,0])]

th = rand()*2*pi
vel = [Swimmers.speed*array([cos(th),sin(th)])]

for l in range(0,20):
    for k in range(0,20):
        pos = np.append(pos,[array([mod(k*0.05 + 0.01*(rand()-0.5) + 0.05,1),
                                  mod(l*0.05 + 0.01*(rand()-0.5) + 0.05,1)])],axis=0)
        th  = rand()*2*pi
        vel = np.append(vel,[Swimmers.speed*array([cos(th),sin(th)])],axis=0)
        
pos_store = list([pos[:,:]])
pos_store = list([np.array(pos)])
scalar_store = list([Swimmers.Meshed()])

count = 1
for plot in range(0,5):
    for k in range(0,1000):
        Swimmers.Update(Swimmers.scalar,pos,vel)
        pos_store.append(np.array(pos))
        scalar_store.append(Swimmers.Meshed())
        aggregations = BD.countblob(BD.blobdetect(255*.50, 255, Swimmers.Meshed()),BD.blobdetect2(255*.50, 255, Swimmers.Meshed()),BD.blobdetect3(255*.50, 255, Swimmers.Meshed()))
        aggreg = np.append(aggreg,[array([count,aggregations])], axis=0)
        count = count + 1

In [ ]:
plt.figure()

plt.plot(aggreg[:,0], aggreg[:,1])   
plt.xlabel('Time')
plt.ylabel('Aggregations')
plt.title(r'Aggregations for $\kappa$: {0:8.2e}, $\beta$: {1:8.2e},'.format(Swimmers.kappa, Swimmers.beta) 
          + '\n' + 
          r'$\lambda$: {0:8.2e}, Deposition Func: {1}'.format(Swimmers.lambda0,'atanDep'))

plt.savefig('atandDep_64_1_1')

# New Animation with Titling Trials

In [ ]:
fig   = plt.figure()
ax    = plt.subplot(1,1,1)
field = ax.pcolormesh(Swimmers.xm,Swimmers.ym,scalar_store[1])
field.set_clim(0,0.08)
dots, = ax.plot([], [], 'ro')
fig.colorbar(field)

def initit():
    dots.set_data([], [])
    return field,dots

def animate(k):
    arr = scalar_store[k]
    arr = arr[:-1, :-1]
    field.set_array(arr.ravel())
    plt.title(r'Frame {0:d}'.format(k))
    plt.suptitle(r'$\kappa$: {0:8.2e}, $\beta$: {1:8.2e}, $\lambda$: {2:8.2e}, Deposition Func: {3}'.format(Swimmers.kappa, Swimmers.beta, Swimmers.lambda0,'linAtanDep'))
    dots.set_data(pos_store[k][:,0],pos_store[k][:,1])

    return field,dots,
    
anim = matplotlib.animation.FuncAnimation(fig,animate,frames=range(0,len(scalar_store),3),
                                          interval=50,blit=False,repeat=True)

# Uncomment if you want to save it to a file.  Requires mencoder or ffmpeg or some writer utility to generate the file.
anim.save('linAtanDep_64_quarter_3.mp4', writer='ffmpeg')
plt.show()